# BERT vs GPT Tokenizer and AutoTokenizer
- 이모지 처리 관점에서 비교
- 모델에 맞는 tokenizer를 자동으로 불러오는 **`AutoTokenizer`**까지 알아보자

## BERT Tokenizer

In [1]:
from transformers import BertTokenizerFast
import torch

이모지를 넣어보자
- BERT: 이모지 데이터 -> [UNK] 처리: 데이터의 완전 손실 -> 디코딩 시 이모지 데이터는 없음
- GPT: 이모지 데이터 -> 이모지를 유니코드 문자 그대로 취급하여 손실 없음 / 이모지를 별도로 전처리해서 토큰화도 가능

In [2]:
# BERT tokenizer 불러오기
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# 예제 text
text = 'This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.'

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
# tokenizer 동작 확인
encoded = tokenizer(
    text,
    padding='max_length', # padding
    truncation=True,      # 너무 길면 자르기
    max_length=30,        # 최대 길이 20 제한
    return_tensors='pt'   # PyTorch 텐서로 변환
)

tokens_bert = tokenizer.tokenize(text)
input_ids_bert = tokenizer.encode(text)
decoded_bert = tokenizer.decode(input_ids_bert)

### BERT Tokenizer 출력 결과

In [9]:
print(f'입력된 문장: {text}')

print("\nTokenized input_ids:")
print(encoded["input_ids"])

print("\nTokenized attention_mask:")
print(encoded["attention_mask"])

print("\nLongTensor 타입 확인:")
print(type(encoded["input_ids"]))

print("\nDecode:")
print(tokenizer.decode(encoded['input_ids'][0]))

print("\nTokens:")
print(tokens_bert)

print("\ninput_ids:")
print(input_ids_bert)

print("\nDecoded input_ids:")
print(decoded_bert)

입력된 문장: This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokenized input_ids:
tensor([[  101,  2023,  3185,  2001, 11757,  7244,  1998,  2018,  2307,  4616,
           999,   999,  1045,  2052,  5791, 16755,  2009,  2000,  3087,   100,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0]])

Tokenized attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0]])

LongTensor 타입 확인:
<class 'torch.Tensor'>

Decode:
[CLS] this movie was incredibly touching and had great performances!! i would definitely recommend it to anyone [UNK]. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

Tokens:
['this', 'movie', 'was', 'incredibly', 'touching', 'and', 'had', 'great', 'performances', '!', '!', 'i', 'would', 'definitely', 'recommend', 'it', 'to', 'anyone', '[UNK]', '.']

input_ids:
[101, 2023, 3185, 2001, 11757, 7244, 1998, 2018, 2307, 4616, 9

|항목|설명|
|:---|:----------------------------|
|`input_ids`|	토크나이징된 BERT용 숫자 시퀀스|
|`attention_mask`|	실제 단어는 1, padding은 0|
|길이|	`max_length=20`으로 제한되었기 때문에 20개|
|`type`|	`<class 'torch.Tensor'>` → LongTensor로 자동 변환|

### Tokenized `input_ids`
```
tensor([
  [101, 2023, 3185, 2001, 11757, 7244, 1998, 2018, 2307, 4616,
   999,  999, 1045, 2052, 5791, 16755, 2009, 2000, 3087, 100,
   1012, 102, 0, 0, 0, 0, 0, 0, 0, 0]
])
```

|토큰 ID|	의미|
|:---|:---|
|101|	[CLS] → 문장의 시작|
|100|	[UNK] → 🤔 이모지를 토크나이저가 모름 → "Unknown" 토큰으로 대체 👉🏻 이모지 2개를 하나로 퉁침|
|1012|	. 마침표|
|102|	[SEP] → 문장의 끝|
|0|	[PAD] → max_length 30에 맞춰 빈자리를 채움|

<br>
<br>

### Tokenized `attention_mask`
```
tensor([
  [1, 1, 1, ..., 1, 1, 0, 0, 0, ..., 0]
])
```
- 1: 실제 단어 또는 특수 토큰 ([CLS], [SEP])
- 0: padding ([PAD] 자리)   
  → 모델이 학습할 때 **0인 위치는 무시**
<br>
<br>

### Decoding Result
```
[CLS] this movie was incredibly touching and had great performances!! i would definitely recommend it to anyone [UNK]. [SEP] [PAD] [PAD] ...
```
- `tokenizer.decode(input_ids)`: token화 된 input tensor를 다시 NL로 변환

## GPT Tokenizer

In [5]:
from transformers import GPT2TokenizerFast

실습 text는 위에 선언한 text 그대로 사용
- 정확한 비교를 위함

In [6]:
# GTP2 토크나이저 불러오기
tokenizer_gpt = GPT2TokenizerFast.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

**GPT는 padding을 지원하지 않는다**   
- GPT는 **문장 생성용 모델**이기 때문에 한 문장씩 **오른쪽으로만 길어지는** 구조
  ```
  GPT는 원래 padding이 필요하지 않다
  ```
- BERT는 문장 분류용으로 **패딩 + 마스킹**이 일반적이다
- GPT2 Token에 pad_token을 명시적으로 추가하면 padding 옵션도 정상 작동하긴 하는데 굳이 느낌임
  ``` python
  # 이런 식으로 하면 됨
  tokenizer_gpt.pad_token = tokenizer_gpt.eos_token
  ```

  <br>

**Summary**   

|원인|해결 방법|
|:---|:---|
|GPT tokenizer는 기본적으로 `pad_token` 없음|	`tokenizer.pad_token = tokenizer.eos_token` 으로 지정|
|padding='max_length' 사용 시 에러 발생 가능|위처럼 `pad_token`을 설정하면 해결|
|BERT와 GPT는 토크나이저 기본 설정이 다름|`AutoTokenizer`를 사용하거나 커스터마이즈 필요|

In [10]:
encoded_gpt = tokenizer_gpt(
    text,
    # padding='max_length', # padding
    truncation=True,      # 너무 길면 자르기
    max_length=30,        # 최대 길이 20 제한
    return_tensors='pt'   # PyTorch 텐서로 변환
)

In [13]:
tokens_gpt = tokenizer_gpt.tokenize(text)
input_ids_gpt = tokenizer_gpt.encode(text)
decoded_gpt = tokenizer_gpt.decode(input_ids_gpt)

### GPT Tokenizer 출력 결과

In [15]:
print(f'입력된 문장: {text}')

print("\nTokenized input_ids:")
print(encoded_gpt["input_ids"])

print("\nTokenized attention_mask:")
print(encoded_gpt["attention_mask"])

print("\nLongTensor 타입 확인:")
print(type(encoded_gpt["input_ids"]))

print("\nDecode:")
print(tokenizer_gpt.decode(encoded_gpt['input_ids'][0]))

print("\nTokens:")
print(tokens_gpt)

print("\ninput_ids:")
print(input_ids_gpt)

print("\nDecoded input_ids:")
print(decoded_gpt)

입력된 문장: This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokenized input_ids:
tensor([[ 1212,  3807,   373,  8131, 15241,   290,   550,  1049, 13289,  3228,
           314,   561,  4753,  4313,   340,   284,  2687, 12520,    97,   242,
          8582,    97,   242,    13]])

Tokenized attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

LongTensor 타입 확인:
<class 'torch.Tensor'>

Decode:
This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokens:
['This', 'Ġmovie', 'Ġwas', 'Ġincredibly', 'Ġtouching', 'Ġand', 'Ġhad', 'Ġgreat', 'Ġperformances', '!!', 'ĠI', 'Ġwould', 'Ġdefinitely', 'Ġrecommend', 'Ġit', 'Ġto', 'Ġanyone', 'ĠðŁ', '¤', 'Ķ', 'ðŁ', '¤', 'Ķ', '.']

input_ids:
[1212, 3807, 373, 8131, 15241, 290, 550, 1049, 13289, 3228, 314, 561, 4753, 4313, 340, 284, 2687, 12520, 97, 242, 8582, 97, 242, 13]

Decoded input_ids:
This m

# AutoTokenizer

In [20]:
from transformers import AutoTokenizer

### AutoTokenizer의 효과
- 코드 재사용성이 높아진다
  ``` python
  from transformers import BertTokenizerFast
  from transformers import GPT2TokenizerFast
  ```
  기존에는 위 코드처럼 tokenizer를 각각 import

- 하단과 같이 코드를 `AutoTokenizer`로 묶게되면, 그에 상응하는 tokenizer를 바로 사용할 수 있다
  - **import를 AutoTokenizer 하나만 하면 되게 되는 것**

### 한계점
- AutoTokenizer는 코드 재사용성을 위함일 뿐
- 테스트 환경에서 사용을 지향하고, 프로덕트 배포 시에는 정확한 Tokenizer를 import해서 사용하는 게 성능/안정성 면에서 좋

In [21]:
tokenizer_auto_gpt = AutoTokenizer.from_pretrained('gpt2')
tokenizer_auto_bert = AutoTokenizer.from_pretrained('bert-base-uncased')

In [40]:
encoded_auto_gpt = tokenizer_auto_gpt(
    text,
    # padding='max_length', # GPT는 padding 지원하지 않는다
    truncation=True,      # 너무 길면 자르기
    max_length=30,        # 최대 길이 20 제한
    return_tensors='pt'   # PyTorch 텐서로 변환
)

encoded_auto_bert = tokenizer_auto_bert(
    text,
    padding='max_length', # padding
    truncation=True,      # 너무 길면 자르기
    max_length=30,        # 최대 길이 20 제한
    return_tensors='pt'   # PyTorch 텐서로 변환
)

### AutoTokenize - BERT Tokenizer

In [39]:
print(f'입력된 문장: {text}')

print("\nTokenized input_ids:")
print(encoded_auto_bert["input_ids"])

print("\nTokenized attention_mask:")
print(encoded_auto_bert["attention_mask"])

print("\nLongTensor 타입 확인:")
print(type(encoded_auto_bert["input_ids"]))

print("\nDecode:")
print(tokenizer_auto_bert.decode(encoded_auto_bert['input_ids'][0]))

print("\nTokens:")
print(tokenizer_auto_bert.tokenize(text))

print("\ninput_ids:")
print(tokenizer_auto_bert.encode(text))

print("\nDecoded input_ids:")
print(tokenizer_auto_bert.decode(encoded_auto_bert["input_ids"][0]))

입력된 문장: This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokenized input_ids:
tensor([[  101,  2023,  3185,  2001, 11757,  7244,  1998,  2018,  2307,  4616,
           999,   999,  1045,  2052,  5791, 16755,  2009,  2000,  3087,   100,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0]])

Tokenized attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0]])

LongTensor 타입 확인:
<class 'torch.Tensor'>

Decode:
[CLS] this movie was incredibly touching and had great performances!! i would definitely recommend it to anyone [UNK]. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

Tokens:
['this', 'movie', 'was', 'incredibly', 'touching', 'and', 'had', 'great', 'performances', '!', '!', 'i', 'would', 'definitely', 'recommend', 'it', 'to', 'anyone', '[UNK]', '.']

input_ids:
[101, 2023, 3185, 2001, 11757, 7244, 1998, 2018, 2307, 4616, 9

### AutoTokenizer - GPT Tokenizer

In [41]:
print(f'입력된 문장: {text}')

print("\nTokenized input_ids:")
print(encoded_auto_gpt["input_ids"])

print("\nTokenized attention_mask:")
print(encoded_auto_gpt["attention_mask"])

print("\nLongTensor 타입 확인:")
print(type(encoded_auto_gpt["input_ids"]))

print("\nDecode:")
print(tokenizer_auto_gpt.decode(encoded_auto_gpt['input_ids'][0]))

print("\nTokens:")
print(tokenizer_auto_gpt.tokenize(text))

print("\ninput_ids:")
print(tokenizer_auto_gpt.encode(text))

print("\nDecoded input_ids:")
print(tokenizer_auto_gpt.decode(encoded_auto_gpt["input_ids"][0]))

입력된 문장: This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokenized input_ids:
tensor([[ 1212,  3807,   373,  8131, 15241,   290,   550,  1049, 13289,  3228,
           314,   561,  4753,  4313,   340,   284,  2687, 12520,    97,   242,
          8582,    97,   242,    13]])

Tokenized attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

LongTensor 타입 확인:
<class 'torch.Tensor'>

Decode:
This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.

Tokens:
['This', 'Ġmovie', 'Ġwas', 'Ġincredibly', 'Ġtouching', 'Ġand', 'Ġhad', 'Ġgreat', 'Ġperformances', '!!', 'ĠI', 'Ġwould', 'Ġdefinitely', 'Ġrecommend', 'Ġit', 'Ġto', 'Ġanyone', 'ĠðŁ', '¤', 'Ķ', 'ðŁ', '¤', 'Ķ', '.']

input_ids:
[1212, 3807, 373, 8131, 15241, 290, 550, 1049, 13289, 3228, 314, 561, 4753, 4313, 340, 284, 2687, 12520, 97, 242, 8582, 97, 242, 13]

Decoded input_ids:
This m